In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import net_utils as nu

In [2]:
### load cifar-10 dataset
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()
onehot_train = nu.convert_to_onehot(y_train, 10)
onehot_test = nu.convert_to_onehot(y_test, 10)

In [ ]:
### visualize some of the images
fig, axes = plt.subplots(2,10,figsize=(20,5))
ctr = 0;
for i in range(2):
    for j in range(10):
        axes[i,j].imshow(np.squeeze(x_train[ctr]))
        axes[i,j].axes.get_xaxis().set_visible(False)
        axes[i,j].axes.get_yaxis().set_visible(False)
        axes[i,j].set_title(str(y_train[ctr]))
        ctr += 1
plt.show()

In [10]:
from basic_cnn import basic_cnn as basic_cnn
import tqdm
import time

class conv_net:
    def __init__(self, im_size, n_chan, n_class, learning_rate):
        # settings
        self.im_size = im_size
        self.n_chan = n_chan
        self.n_class = n_class
        self.learning_rate = learning_rate

        # placeholders for inputs and ouputs
        self.inputs = tf.placeholder(tf.float32, (None, im_size, im_size, n_chan), name="inputs")
        self.targets = tf.placeholder(tf.float32, (None, n_class), name="targets")
        pred, cost = basic_cnn(self.inputs, self.targets)
        self.pred = pred
        self.cost = cost
        
        # optimization function
        vars_to_minimize = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='conv_net')
        self.opt = tf.train.AdamOptimizer(self.learning_rate).minimize(cost, var_list=vars_to_minimize)
        
        # metrics
        self.auc_list = []
        self.auc_op_list = []
        self.f1s_list = []
        self.f1s_op_list = []
        
        for i in range(self.n_class):
            auc, auc_op = tf.metrics.auc(self.targets[:,i], self.pred[:,i])
            self.auc_list.append(auc)
            self.auc_op_list.append(auc_op)
            f1s, f1s_op =  tf.contrib.metrics.f1_score(self.targets[:,i], self.pred[:,i])
            self.f1s_list.append(f1s)
            self.f1s_op_list.append(f1s_op)
            
        self.avg_auc = tf.add_n(self.auc_list) / n_class
        self.avg_f1s = tf.add_n(self.f1s_list) / n_class
        
    def parameter_count(self, full=False):
        vars_to_minimize = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='conv_net')
        nu.parameter_count(vars_to_minimize, full = False)
        
    def train_on_batch(self, inputs, targets, mb_sz = 32):
        data_count = inputs.shape[0]

        for i in range(0, data_count, mb_sz):
            i_s = i
            i_e = np.minimum(data_count, i + mb_sz)
            sess_out = self.opt
            feed_dict={self.inputs: inputs[i_s:i_e], self.targets: targets[i_s:i_e]}
            sess.run(sess_out, feed_dict=feed_dict)
            time.sleep(0.01)
            
    def run_on_batch(self, inputs, targets, mb_sz = 32):
        data_count = inputs.shape[0]
        pred = np.zeros((data_count, self.n_class))
        cost = 0

        for i in range(0, data_count, mb_sz):
            i_s = i
            i_e = np.minimum(data_count, i + mb_sz)
            
            sess_out = [self.pred, self.cost]
            feed_dict={self.inputs: inputs[i_s:i_e], self.targets: targets[i_s:i_e]}
            results = sess.run(sess_out, feed_dict=feed_dict)
            pred[i_s:i_e] = results[0]
            cost += (results[1] / (i_e-i_s))
            time.sleep(0.01)
            
        return pred, cost
    
    def run_on_batch_wm(self, inputs, targets, mb_sz = 32):
        data_count = inputs.shape[0]
        pred = np.zeros((data_count, self.n_class))
        cost = 0
        
        auc_variables = [ v for v in tf.local_variables() if v.name.startswith( 'auc' ) ]
        sess.run(tf.initialize_variables(auc_variables))
        f1_variables = [ v for v in tf.local_variables() if v.name.startswith( 'f1' ) ]
        sess.run(tf.initialize_variables(f1_variables))
        
        for i in range(0, data_count, mb_sz):
            i_s = i
            i_e = np.minimum(data_count, i + mb_sz)
            
            sess_out = [self.pred, self.cost]
            sess_out.extend(self.auc_op_list)
            sess_out.extend(self.f1s_op_list)
            feed_dict={self.inputs: inputs[i_s:i_e], self.targets: targets[i_s:i_e]}
            results = sess.run(sess_out, feed_dict=feed_dict)
            pred[i_s:i_e] = results[0]
            cost += (results[1] / (i_e-i_s))
            time.sleep(0.01)
            
        sess_out = [self.avg_auc, self.avg_f1s]
        results = sess.run(sess_out, feed_dict={})
        auc = results[0]
        f1s = results[1]
        
        return pred, cost, auc, f1s

In [ ]:
### clear the graph
tf.reset_default_graph()
### build the network
net = conv_net(im_size = 32, n_chan = 3, n_class = 10, learning_rate = 1e-3)
net.parameter_count()
### initialze the session and variables
sess = tf.Session()
sess.run(tf.global_variables_initializer()) 

epochs = 20
for e in range(epochs):
    nu.shuffle_together(x_train, onehot_train)
    net.train_on_batch(x_train, onehot_train)
#     _, trn_cost = net.run_on_batch(x_train, onehot_train)
#     _, tst_cost = net.run_on_batch(x_test, onehot_test)
    _, trn_cost, trn_auc, trn_f1s = net.run_on_batch_wm(x_train, onehot_train)
    _, tst_cost, tst_auc, tst_f1s = net.run_on_batch_wm(x_test, onehot_test)
    
    print("Epoch: {}/{}...".format(e+1, epochs),
          "trn loss: {:.5f}".format(trn_cost),
          "tst loss: {:.5f}".format(tst_cost))
    print("            ...".format(e+1, epochs),
          "trn auc:  {:.5f}".format(trn_auc),
          "tst auc:  {:.5f}".format(tst_auc))
    print("            ...".format(e+1, epochs),
          "trn f1s:  {:.5f}".format(trn_f1s),
          "tst f1s:  {:.5f}".format(tst_f1s))

trainable parameters: 270270
Instructions for updating:
Use `tf.variables_initializer` instead.
Epoch: 1/20... trn loss: 73.20692 tst loss: 15.07689
            ... trn auc:  0.87608 tst auc:  0.86892
            ... trn f1s:  0.51047 tst f1s:  0.49919
Epoch: 2/20... trn loss: 60.07050 tst loss: 12.99995
            ... trn auc:  0.91213 tst auc:  0.89777
            ... trn f1s:  0.58689 tst f1s:  0.55841
Epoch: 3/20... trn loss: 54.84756 tst loss: 12.42193
            ... trn auc:  0.92673 tst auc:  0.90719
            ... trn f1s:  0.62568 tst f1s:  0.58767
Epoch: 4/20... trn loss: 51.95334 tst loss: 12.32841
            ... trn auc:  0.93417 tst auc:  0.90999
            ... trn f1s:  0.64581 tst f1s:  0.59429
Epoch: 5/20... trn loss: 50.38515 tst loss: 12.33060
            ... trn auc:  0.93942 tst auc:  0.91274
            ... trn f1s:  0.66300 tst f1s:  0.60213
Epoch: 6/20... trn loss: 48.91837 tst loss: 12.40751
            ... trn auc:  0.94206 tst auc:  0.91200
            ..